In [1]:
import xmltodict
import math
import urllib.request
import itertools
import pandas as pd
import pickle
from sklearn.utils import shuffle
import time

In [10]:
from urllib.request import urlopen
# the original csv from https://raw.githubusercontent.com/beefsack/bgg-ranking-historicals/master/2017-02-21.csv
# The ID's where used in API calls to retrieve the game reviews
link="https://raw.githubusercontent.com/beefsack/bgg-ranking-historicals/master/2020-08-19.csv"
f = urlopen(link)
games = pd.read_csv(f)
games.describe()
games.sort_values('Users rated',ascending=False,inplace=True)
games[:10]
games.to_csv('2020-08-19.csv')

In [3]:
def gen_batches():
    num_batches = len(games)//100
    for b in range(num_batches):
        result = slice(b*100,(b+1)*100)
        if b*100>-1:
            yield result
    yield slice((b+1)*100,len(games)+1)



In [4]:
def number_rating_pages(game_id):
    url = 'https://www.boardgamegeek.com/xmlapi2/thing?id='+str(game_id)+'&ratingcomments=1'
    u = urllib.request.urlopen(url).read()#The url you want to open
    doc = xmltodict.parse(u)
    return math.ceil(int(doc['items']['item']['comments']['@totalitems'])/100)



In [5]:
def get_url_result(ids): # given a url of an object and a page, return list of dicts with the comments
    url = 'https://www.boardgamegeek.com/xmlapi2/thing?id='+str(ids)+'&stats=1'
    #url += '&&page='+str(p)
    u = urllib.request.urlopen(url).read()#The url you want to open
    doc = xmltodict.parse(u)
    return doc 



In [6]:
def process_game_result(url_result):
    games = []
    for g in url_result['items']['item']:
        games.append(process_game(g))
    return games

In [7]:
def process_game(a):
    res = {}
    for k in a.keys():
        if k == '@type': res['type']=a[k]
        if k == '@id': res['id']=a[k]
        if k == 'thumbnail': res['thumbnail']=a[k]
        if k == 'image': res['image']=a[k]
        if k == 'name':
            if isinstance(a[k], list):
                for item in a[k]:
                    if item['@type'] in res.keys():
                        res[item['@type']].append(item['@value'])
                    else:
                        res[item['@type']] = [item['@value']]  
            else:
                res[a[k]['@type']] = [a[k]['@value']]


        if k == 'description': res[k]=a[k]
        if k == 'yearpublished': res[k]=a[k]['@value']
        if k == 'minplayers': res[k]=a[k]['@value'] 
        if k == 'maxplayers': res[k]=a[k]['@value'] 
        if k == 'playingtime': res[k]=a[k]['@value']        
        if k == 'minplaytime': res[k]=a[k]['@value']
        if k == 'maxplaytime': res[k]=a[k]['@value']
        if k == 'minage': res[k]=a[k]['@value']
        if k == 'link': 
            for item in a[k]:
                if item['@type'] in res.keys():
                    res[item['@type']].append(item['@value'])
                else:
                    res[item['@type']] = [item['@value']]
        if k == 'statistics':

            for kk in a[k]['ratings'].keys():
                if kk == 'ranks':
                    if isinstance(a[k]['ratings']['ranks']['rank'], list):
                        for r in a[k]['ratings']['ranks']['rank']:
                            res[r['@friendlyname']] = r['@value']    
                    else:
                        res[a[k]['ratings']['ranks']['rank']['@friendlyname']] = a[k]['ratings']['ranks']['rank']['@value']    
                else:
                    res[kk] = a[k]['ratings'][kk]['@value']
        if k == 'poll': 
            for p in a[k]:
                if p['@name'] == 'suggested_numplayers':
                    if 'results' in p.keys():
                        res['suggested_num_players']= p['results'] # Added this line
                    
                        # if isinstance(p['results'], list):

                        #     for num in p['results']:
                        #         suggestion = []
                        #         if isinstance(num['result'], list):
                        #             for s in num['result']: suggestion.append(s['@numvotes'])
                        #             res['suggested_num_players='+num['@numplayers']]=suggestion
                        #         else:
                        #             res['suggested_num_players='+num['result']['@numplayers']]=num['result']['@numvotes']
                        # else:
                        #     if 'result' in p['results'].keys():
                        #         if isinstance(p['results']['result'], list):
                        #             suggestion = []
                        #             for s in p['results']['result']: suggestion.append(s['@numvotes'])
                        #             res['suggested_num_players='+p['results']['@numplayers']]=suggestion
                        #         else:
                        #             res['suggested_num_players='+p['results']['result']['@numplayers']]=p['results']['result']['@numvotes']
                if p['@name'] == 'suggested_playerage':
                    if 'results' in p.keys():
                        res['suggested_playerage']=p['results']['result']
                        #                         if isinstance(p['results']['result'], list):                    
                        #                             for l in p['results']['result']: res['suggested_playerage='+l['@value']]=l['@numvotes']
                        #                         else:
                        #                             res['suggested_playerage='+p['results']['result']]=p['results']['result']['@numvotes']

                if p['@name'] == 'language_dependence':
                    if 'results' in p.keys():
                        res['suggested_language_dependence']=p['results']['result']
                        #                         if isinstance(p['results']['result'], list):
                        #                             for l in p['results']['result']: res['suggested_language_dependence='+l['@value']]=l['@numvotes']
                        #                         else:
                        #                             res['suggested_language_dependence='+p['results']['result']['@value']]=p['results']['result']['@numvotes']

    return res

In [8]:
game_list = []
game_generator = gen_batches()

# get all games, but do this per batch of 100 games
for batch in game_generator: 
    for i in range(5):
        try:
            print('new batch,', batch)
            # list of games
            games_in_batch = list(games['ID'][batch])
            
            print('next query for {} games'.format(len(games_in_batch)))
            # convert list of current games to url input
            ids = ','.join(str(g) for g in games_in_batch)
            #query API and parse to dictionary
            url_result = get_url_result(ids)
            print('data retrieved')
            #print('to remove',removelist)
            game_list += process_game_result(url_result)
            time.sleep(5)
            break
        except Exception as e:
            print (e,i,batch)
            time.sleep(30)
with open('games.pickle', 'wb') as handle:
    pickle.dump(game_list, handle, protocol=pickle.HIGHEST_PROTOCOL)
print('all done')

new batch, slice(0, 100, None)
next query for 100 games
data retrieved
new batch, slice(100, 200, None)
next query for 100 games
data retrieved
new batch, slice(200, 300, None)
next query for 100 games
data retrieved
new batch, slice(300, 400, None)
next query for 100 games
data retrieved
new batch, slice(400, 500, None)
next query for 100 games
data retrieved
new batch, slice(500, 600, None)
next query for 100 games
data retrieved
new batch, slice(600, 700, None)
next query for 100 games
data retrieved
new batch, slice(700, 800, None)
next query for 100 games
data retrieved
new batch, slice(800, 900, None)
next query for 100 games
data retrieved
new batch, slice(900, 1000, None)
next query for 100 games
data retrieved
new batch, slice(1000, 1100, None)
next query for 100 games
data retrieved
new batch, slice(1100, 1200, None)
next query for 100 games
data retrieved
new batch, slice(1200, 1300, None)
next query for 100 games
data retrieved
new batch, slice(1300, 1400, None)
next query 

In [9]:
games= pd.DataFrame(game_list)
pd.options.display.max_rows = 200
games['primary']=games['primary'].map(lambda x: str(x)[2:-2])
games.to_csv(r'games_detailed_info.csv')
games.head().T

,0,1,2,3,4
type,boardgame,boardgame,boardgame,boardgame,boardgame
id,30549,822,13,68448,36218
thumbnail,https://cf.geekdo-images.com/thumb/img/HEKrtpT...,https://cf.geekdo-images.com/thumb/img/kqE4YJS...,https://cf.geekdo-images.com/thumb/img/g8LvJsd...,https://cf.geekdo-images.com/thumb/img/Grz-qM9...,https://cf.geekdo-images.com/thumb/img/iPlTR5c...
image,https://cf.geekdo-images.com/original/img/j-pf...,https://cf.geekdo-images.com/original/img/o4p6...,https://cf.geekdo-images.com/original/img/A-0y...,https://cf.geekdo-images.com/original/img/3DP_...,https://cf.geekdo-images.com/original/img/oN8C...
primary,Pandemic,Carcassonne,Catan,7 Wonders,Dominion
alternate,"[EPIZOotic, Pandemia, Pandemia 10 Aniversario,...","[Carcassonne Jubilee Edition, Carcassonne: Plu...","[CATAN, Catan (Колонизаторы), Catan telepesei,...","[7 csoda, 7 Cudów Świata, 7 divů světa, 7 чуде...","[Dominion: Basisspiel, Dominion: In naam van d..."
description,"In Pandemic, several virulent diseases have br...",Carcassonne is a tile-placement game in which ...,"In Catan (formerly The Settlers of Catan), pla...",You are the leader of one of the 7 great citie...,"&quot;You are a monarch, like your parents bef..."
yearpublished,2008,2000,1995,2010,2008
minplayers,2,2,3,2,2
maxplayers,4,5,4,7,4
